In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib as mpl
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

mpl.rc('font',**{'family':'serif','serif':['serif']})
mpl.rc('text', usetex=True)
import fitsio as fio

import copy

import xpipe.tools.catalogs as catalogs
import xpipe.paths as paths
import xpipe.xhandle.parbins as parbins
import xpipe.xhandle.xwrap as xwrap
import xpipe.tools.selector as selector
import xpipe.xhandle.shearops as shearops
import xpipe.xhandle.pzboost as pzboost
import xpipe.tools.y3_sompz as sompz
import xpipe.tools.mass as mass
import xpipe.tools.visual as visual


from importlib import reload
import pickle


import astropy.cosmology as cosmology
# this is just the default cosmology
cosmo = cosmology.FlatLambdaCDM(Om0=0.3, H0=70)

# we make sure the correct config file is loaded here, it will let us automatically now what type of files 
# were / will be produced, and where they will be placed
paths.update_params("/home/moon/vargatn/DES/PROJECTS/xpipe/settings/params_y3lwb_meta.yml")

reading DEFAULTS from default_params.yml
updating params from: /home/moon/vargatn/DES/PROJECTS/xpipe/settings/params.yml
updating params from: /home/moon/vargatn/DES/PROJECTS/xpipe/settings/params_y3rm-stack_meta.yml
updating params from: /home/moon/vargatn/DES/PROJECTS/xpipe/settings/params_y3lwb_meta.yml

***********************

running in full mode

***********************



# Creating lens fits table

xpipe takes a fits table as an input, with the columns specified in the config file

In [2]:
# first we load the CMASS data
tmp = np.loadtxt("DESY3_CMASS.dat")[:, (0, 1, 6, 7)]
cmass = pd.DataFrame(data=tmp, columns=("RA", "DEC", "WSYS", "Z"))
cmass["ID"] = np.arange(len(cmass), dtype=int) + 1e6
oname = "/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/lenscat/LWB_DESY3_CMASS.fits"
# fio.write(oname, cmass.to_records(), clobber=True)

In [3]:
# # then the LOWZ data
tmp = np.loadtxt("DESY3_LOWZ.dat")[:, (0, 1, 6, 7)]
lowz = pd.DataFrame(data=tmp, columns=("RA", "DEC", "WSYS", "Z"))
lowz["ID"] = np.arange(len(lowz), dtype=int)
oname = "/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/lenscat/LWB_DESY3_LOWZ.fits"
# fio.write(oname, lowz.to_records(), clobber=True)

In [4]:
# # and concatenate them, this is going to be useful to automating the lensing measurement
# # file names and paths will be taken care of automatically
allgal = pd.concat((lowz, cmass), ignore_index=True)
oname = "/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/lenscat/LWB_DESY3_ALL.fits"
fio.write(oname, allgal.to_records(), clobber=True)

In [5]:
# now extract the len weights, these are not used yet we can pass them on the post-processing stage
weights = pd.DataFrame()
weights["ww"] = allgal["WSYS"]
weights["ID"] = allgal["ID"]

# Running xpipe

* `python mkbins.py --norands --params=params_y3lwb_meta.yml`  this prepares the Jackknife regions and writes the lenses to the proper input format from the fits, table

* `python xshear_metacal.py --norands --params=params_y3lwb_meta.yml --runall` this runs the shear measurement 5 times for metacal, each JK region is processed separately, so for 1 profile up to ~500 different cores can be used simultaneously (assuming ~100 JK patches)

# Postprocessing the lensing signal

The Y3 xpipe setup measures shear, so we need to also access the DES Y3 photo-z cats

In [6]:
main_file_path = "/e/ocean1/users/vargatn/DESY3/Y3_mastercat_03_31_20.h5"
src = sompz.sompz_reader(main_file_path)
src.build_lookup()

In [7]:
# the output of xshear_metacal.py is in the following files.
# flist is the lenses
# rlist is the random points
flist, flist_jk, rlist, rlist_jk = parbins.get_file_lists(paths.params, paths.dirpaths)

In [43]:
ms_opt=[np.array([-0.02,-0.024,-0.037]),np.array([-0.024,-0.037]),np.array([-0.024,-0.037]),np.array([-0.037,])]
optsbins=[(1,2,3),(2,3),(2,3),(3,)]

ACPs = []
ACP_ms = []
ACP_optms = []
for i, fname in enumerate(flist):
    mfac_opt=1/(1+ms_opt[i])
#     print(mfac, mfac_opt)
    
    ACP = shearops.AutoCalibrateProfile([fname,], flist_jk[i], src, xlims=(0.01, 30), sbins=optsbins[i])
    ACP.get_profiles(ismeta=True, id_key="ID", z_key="Z", weights=weights, mfactor_sbins=mfac_opt) #
    ACP_optms.append(ACP)

loading source bin 1
loading source bin 2
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1clust_qbin-0.fits
loading source bin 2
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1clust_qbin-1.fits
loading source bin 2
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1clust_qbin-2.fits
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1clust_qbin-3.fits


In [9]:
ACP_optm_rands = []
for i, rname in enumerate(rlist):
    mfac_opt=1/(1+ms_opt[i])
    
#     # This is a hotfix for the catalog matching bug... only need to be run once if you encounter obviously incorrect sigma crits with random points
#     fname = "/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1rand_qbin-"+str(i)+".fits"
#     tmp = fio.read(fname)
#     tmp["ID"] = tmp["index"]
#     fio.write(fname, tmp, clobber=True)    

    # processing randoms
    ACP = shearops.AutoCalibrateProfile([rname,], rlist_jk[i], src, xlims=(0.01, 30), sbins=optsbins[i])
    ACP.get_profiles(ismeta=True, id_key="ID", z_key="Z", mfactor_sbins=mfac_opt) #
    ACP_optm_rands.append(ACP)

loading source bin 1
loading source bin 2
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1rand_qbin-0.fits
loading source bin 2
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1rand_qbin-1.fits
loading source bin 2
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1rand_qbin-2.fits
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v02/lwb-y3-lowz+cmass_v02_y1rand_qbin-3.fits


# Calculating boost factors from P(z) decomposition

xpipe automatically saves a fraction of source-lens pairs, and these are used to estimate the boost factors

In [10]:
# # !!! Uncomment this block and run once !!!
# optsbins=[(1,2,3),(2,3),(2,3),(3,)]
# # # For each redshift bin
# for i in np.arange(4):
#     # First we need to match the source id with the BPZ photot-z
#     smb = pzboost.SOMBoost(src, [flist_jk[i],], sbins=optsbins[i])
#     smb.get_pair_datas(pair_outpath="lwb-y3-lowz+cmass_v02_y1clust_optsbins_qbin-"+str(i)+"_pairs.h5") 
#     #     # this could take a few minutes or so
#     #     # It saves the matched version of the pairs to the specified file name    

In [44]:
# Then run this
optsbins=[(1, 2, 3),(2, 3),(2, 3),(3,)]
smbs = []
ACPs_boosted = []
for i in np.arange(4):
    # Load the pairs files pre-processed above
    smb = pzboost.SOMBoost(src, [flist_jk[i],], sbins=optsbins[i],
                           pairs_to_load="lwb-y3-lowz+cmass_v02_y1clust_optsbins_qbin-"+str(i)+"_pairs.h5")
    smb.prep_boost(bins_to_use=np.arange(10)) # these are the indices of the radial bins to be used
    smb.get_boost(npdf=9) # use the inner 9 radial bins (and the last as the reference PDF)
    smbs.append(smb)
    ACP = copy.copy(ACP_optms[i])
    ACP.add_boost(smb)
#     ACP.profile.multiply((ACP.fcl + 1)[:, np.newaxis])
    ACPs_boosted.append(ACP)

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
0 0
0 1


/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/scipy/optimize/_lsq/common.py:453: RuntimeWarning: underflow encountered in nextafter
  x_new[lower_mask] = np.nextafter(lb[lower_mask], ub[lower_mask])
/home/moon/vargatn/DES/PROJECTS/xpipe/xpipe/xhandle/pzboost.py:975: RuntimeWarning: underflow encountered in multiply
  self.mixarr[i, :] = amps[i] * garr + (1. - amps[i]) * self.refpdf
/home/moon/vargatn/anaconda3/envs/main3/lib/python3.8/site-packages/scipy/optimize/_lsq/trf.py:263: RuntimeWarning: underflow encountered in multiply
  g_norm = norm(g * v, ord=np.inf)


0 2


/home/moon/vargatn/DES/PROJECTS/xpipe/xpipe/xhandle/shearops.py:989: RuntimeWarning: underflow encountered in multiply
  tmps.append(_amp * self.scinvs[i])


(9,)
1
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
0 0
0 1
(9,)
1
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
0 0
0 1
(9,)
1
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]
0 0
(9,)
1


In [12]:
# calculate stack-wise subtraction
profiles_subtracted = []
for i in np.arange(4):
    prof1 = copy.copy(ACPs_boosted[i].profile)
#     prof1 = copy.copy(ACP_optms[i].profile)
    prof2 = copy.copy(ACP_optm_rands[i].profile)
#     prof = prof1.profile.composite
    prof1.composite(prof2, operation="-")
    profiles_subtracted.append(prof1)

In [47]:
def write_profile(prof, path):
    """saves DeltaSigma and covariance in text format"""

    # Saving profile
    profheader = "R [Mpc]\tDeltaSigma_t [M_sun / pc^2]\tDeltaSigma_t_err [M_sun / pc^2]\tDeltaSigma_x [M_sun / pc^2]\tDeltaSigma_x_err [M_sun / pc^2]"
    res = np.vstack((prof.rr, prof.dst, prof.dst_err, prof.dsx, prof.dsx_err)).T
    fname = path + "_profile.dat"
    print("saving:", fname)
    np.savetxt(fname, res, header=profheader)

    # Saving covariance
    np.savetxt(path + "_dst_cov.dat", prof.dst_cov)
    np.savetxt(path + "_dsx_cov.dat", prof.dsx_cov)

In [48]:
# tmp_profs = copy.deepcopy(profiles)
fnames = []
for i in np.arange(4):
#     for s in np.arange(3):
#         scinv = scinvs[i][s+1]
#         prof = tmp_profs[i][s]
#         prof.multiply(1 / scinv)
        fname = "LWB_lowz+cmass_boost_rand-subtr_zbin"+str(i)+"_combined"
        fnames.append(fname)
        prof = profiles_subtracted[i]
        write_profile(prof, fname)

saving: LWB_lowz+cmass_boost_rand-subtr_zbin0_combined_profile.dat
saving: LWB_lowz+cmass_boost_rand-subtr_zbin1_combined_profile.dat
saving: LWB_lowz+cmass_boost_rand-subtr_zbin2_combined_profile.dat
saving: LWB_lowz+cmass_boost_rand-subtr_zbin3_combined_profile.dat
